# 根据要求导出`WRF-CMAQ`模拟数据
2、	四个国控点的数据。各指标如O3浓度、PM2.5浓度、NO2浓度、各过程贡献、各来源贡献和敏感性指标（HNO3/H2O2比值）的时间序列，可以存成一个excel表格，例如第一列是时间（小时），第二列是O3浓度，第三列….。

---
*@author: Evan*\
*@date: 2023-05-12*

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from scipy.io import savemat

from matplotlib import rcParams
config = {
    "font.family":'Times New Roman',
    "mathtext.fontset":'stix',
    "font.serif": ['SimSun'],
}
rcParams.update(config)

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append('../src/')
from findpoint import findpoint
import ModelEvaLib as me

In [10]:
chemFile = xr.open_dataset('F:/Data/case_anqing/April/April_chem.nc')
metFile  = xr.open_dataset('F:/Data/case_anqing/April/April_met.nc')
isam = xr.open_dataset('F:/Data/case_anqing/April/April_ISAM1.nc')
pa=xr.open_dataset('F:/Data/case_anqing/April/April_PA1.nc')

lat = chemFile.latitude
lon = chemFile.longitude

ncfile = chemFile.O3
O3   = chemFile.O3[:,0,:,:]
NO2  = chemFile.NO2[:,0,:,:]
PM25 = chemFile.PM25[:,0,:,:]
ht   = chemFile.HT[:,0,:,:]

HADV = pa.HADV_O3[:,0,:,:]
ZADV = pa.ZADV_O3[:,0,:,:]
HDIF = pa.HDIF_O3[:,0,:,:]
VDIF = pa.VDIF_O3[:,0,:,:]
DDEP = pa.DDEP_O3[:,0,:,:]
CHEM = pa.CHEM_O3[:,0,:,:]

AQ  = isam.O3_AQ[:,0,:,:]
HF  = isam.O3_HF[:,0,:,:]
TL  = isam.O3_TL[:,0,:,:]
CZ  = isam.O3_CZ[:,0,:,:]
JJ  = isam.O3_JJ[:,0,:,:]
OTH = isam.O3_OTH[:,0,:,:]
BCO = isam.O3_BCO[:,0,:,:]
ICO = isam.O3_ICO[:,0,:,:]

HNO3 = pa.HNO3prod[:,0,:,:]
H2O2 = pa.H2O2prod[:,0,:,:]

In [6]:
siteloc = pd.read_excel('F:/Data/case_anqing/April/obsdata/安庆市站点坐标.xlsx',header=0,index_col=0)

sites = ['安庆大学','市人大','环科院','马山宾馆','安庆政务中心']

lono = siteloc[siteloc.index.isin(sites)]['经度']
lato = siteloc[siteloc.index.isin(sites)]['纬度']

lono

站点名称
安庆大学      116.9898
市人大       117.0597
环科院       117.0807
马山宾馆      117.0331
安庆政务中心    117.1150
Name: 经度, dtype: float64

In [13]:
x_index=[]
y_index=[]

for name in sites:
    out_x, out_y = findpoint(lono[name],lato[name],ncfile)
    x_index.append(out_x)
    y_index.append(out_y)
O3_sel = O3.isel(x=x_index, y=y_index)
NO2_sel  = NO2.isel(x=x_index, y=y_index)
PM25_sel = PM25.isel(x=x_index, y=y_index)
htsel    = ht.isel(x=x_index, y=y_index)

HADV_sel  = HADV.isel(x=x_index, y=y_index)
ZADV_sel  = ZADV.isel(x=x_index, y=y_index)
HDIF_sel  = HDIF.isel(x=x_index, y=y_index)
VDIF_sel  = VDIF.isel(x=x_index, y=y_index)
DDEP_sel  = DDEP.isel(x=x_index, y=y_index)
CHEM_sel  = CHEM.isel(x=x_index, y=y_index)

HNO3_sel = HNO3.isel(x=x_index, y=y_index)
H2O2_sel = H2O2.isel(x=x_index, y=y_index)

AQ_sel  = AQ.isel(x=x_index, y=y_index)
HF_sel  = HF.isel(x=x_index, y=y_index)
TL_sel  = TL.isel(x=x_index, y=y_index)
CZ_sel  = CZ.isel(x=x_index, y=y_index)
JJ_sel  = JJ.isel(x=x_index, y=y_index)
OTH_sel = OTH.isel(x=x_index, y=y_index)
BCO_sel = BCO.isel(x=x_index, y=y_index)
ICO_sel = ICO.isel(x=x_index, y=y_index)

In [17]:
for n in range(5):
    df_site = pd.DataFrame(
        index=pd.date_range('2023-04-01T00','2023-04-30T23',freq='H'),
        data={
            'O3':O3_sel[:,n,n].data,
            'NO2':NO2_sel[:,n,n].data,
            'PM25':PM25_sel[:,n,n].data,
            'HADV':HADV_sel[:,n,n].data,
            'ZADV':ZADV_sel[:,n,n].data,
            'HDIF':HDIF_sel[:,n,n].data,
            'VDIF':VDIF_sel[:,n,n].data,
            'DDEP':DDEP_sel[:,n,n].data,
            'CHEM':CHEM_sel[:,n,n].data,
            'HNO3/H2O2':HNO3_sel[:,n,n].data/H2O2_sel[:,n,n].data,
            'AQ':AQ_sel[:,n,n].data,
            'HF':HF_sel[:,n,n].data,
            'TL':TL_sel[:,n,n].data,
            'CZ':CZ_sel[:,n,n].data,
            'JJ':JJ_sel[:,n,n].data,
            'OTH':OTH_sel[:,n,n].data,
            'BCO':BCO_sel[:,n,n].data,
            'ICO':ICO_sel[:,n,n].data,
        }
    )
    df_site.to_excel(f'D:/Download/df_{n}.xlsx')